In [46]:
import pandas as pd
import requests

In [47]:
dfpostpath = "postAITS.csv"
followerpath  = "followers1.csv"

In [48]:
from retrieve_post_by_userid import RetrievePostByUserId
r = RetrievePostByUserId(sessionid="")

In [49]:

import re
def get_threads_userpk(username:str):
    
    headers = {
        "Accept": "*/*",
        "Accept-Language": "it-IT,it;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://www.threads.net",
        "Priority": "u=1, i",
        "Referer": f"https://www.threads.net/@{username}",
        "Sec-CH-UA": '"Not(A:Brand";v="99", "Brave";v="133", "Chromium";v="133"',
        "Sec-CH-UA-Full-Version-List": '"Not(A:Brand";v="99.0.0.0", "Brave";v="133.0.0.0", "Chromium";v="133.0.0.0"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Model": "",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-CH-UA-Platform-Version": '"19.0.0"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-ASBD-ID": "129477",
        "X-FB-LSD": "AVrgJ17PSUU"
    }
    # cookies = {
    #         "sessionid": sessionid
    # }
    data = {
        "client_previous_actor_id": "",
        "route_url": f"/@{username}",
        "routing_namespace": "barcelona_web",
        "__user": "0",
        "__a": "1",
        "__req": "m",
        "__hs": "20128.HYP:barcelona_web_pkg.2.1...0",
        "dpr": "1",
        "__comet_req": "29",
        "lsd": "AVrgJ17PSUU",
    }
    response = requests.post(
            url = "https://www.threads.net/ajax/navigation/",
            headers=headers,
            data = data
                # cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

id = get_threads_userpk("mlssfshn")
print(id)

187264532


In [50]:
df = pd.read_csv(followerpath)
df

,thread_user_pk,thread_username,thread_follower_pk,thread_follower_username,follower_count
0,6.307307e+10,aliencoremuzik,7.072615e+10,arts.wh0re,NaN
1,6.307307e+10,aliencoremuzik,6.088223e+10,cc.jayyyy,NaN
2,6.307307e+10,aliencoremuzik,5.538650e+10,aliquashady999,NaN
3,6.307307e+10,aliencoremuzik,7.064831e+10,dsombk,NaN
4,6.307307e+10,aliencoremuzik,5.223278e+10,mojito.mike21,NaN
...,...,...,...,...,...
10324,6.459065e+10,jodz_is_it_,7.024064e+10,kiamichii,NaN
10325,6.459065e+10,jodz_is_it_,6.360179e+10,johnny_soda,NaN
10326,6.459065e+10,jodz_is_it_,7.022145e+10,orionsvoice,NaN
10327,6.459065e+10,jodz_is_it_,6.305567e+10,princessfangirl2.0,NaN


In [51]:
def analize_post(data_dict,thread_user_pk):
    extracted_posts = []
    for edge in data_dict.get("data", {}).get("mediaData", {}).get("edges", []):
        node = edge.get("node", {})
        for item in node.get("thread_items", []):
            post = item.get("post", {})
            extracted = {
                "id": post.get("id"),
                "post_pk": post.get("pk"),
                "caption_text": post.get("caption", {}).get("text") if post.get("caption") else None,
                "like_count": post.get("like_count"),
                "taken_at": post.get("taken_at"),
                "username": post.get("user", {}).get("username"),
                "user_pk": post.get("user", {}).get("pk"),
                "quote_count": post.get("text_post_app_info", {}).get("quote_count"),
                "repost_count": post.get("text_post_app_info", {}).get("repost_count"),
                "reshare_count": post.get("text_post_app_info", {}).get("reshare_count"),
                "thread_user_pk": thread_user_pk,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }
            extracted_posts.append(extracted)
            
            
    return extracted_posts


In [52]:

dfpost = pd.read_csv(dfpostpath)
dfpost
user_processed = dfpost["thread_user_pk"].astype(str).unique()
len(user_processed)

10132

In [53]:
user_processed =list(user_processed)
user_processed.append("63436262560.0")
user_processed.append("65190716943.0")
user_processed.append("66891631140.0")
user_processed.append("54971006231.0")
user_processed.append("63414786986.0")



In [54]:
previus = ""
for idx ,row in df.iterrows():
    if str(row["thread_follower_pk"]) == previus:
        print("Processed ", idx, "on ", len(df))
        continue
    if not str(row["thread_follower_pk"]) in user_processed:
        previus = row["thread_follower_pk"]
        print(row["thread_follower_pk"], " ",row["thread_follower_username"])
        try:
            id = get_threads_userpk(row["thread_follower_username"])
        except:
            if "46962366031" == str(get_threads_userpk("shawnecesellsatl")):
                user_processed.append(row["thread_follower_pk"])
                print("Processed ", idx, "on ", len(df))
                continue
        posts = r.retrieve_thread_by_userid(userid=str(id))
        if posts is None or  posts["data"] is None or posts["data"]["mediaData"] is None:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        elif not id or not posts["data"]["mediaData"]["edges"]:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        else:
            temp_df = pd.DataFrame(analize_post(posts,row["thread_follower_pk"]))
        user_processed.append(str(row["thread_follower_pk"]))
        temp_df.to_csv(dfpostpath, mode='a', header=False, index=False)
        print("Processed ", idx, "on ", len(df))
    else:
        print("Processed ", idx, "on ", len(df))

Processed  0 on  10329
Processed  1 on  10329
Processed  2 on  10329
Processed  3 on  10329
Processed  4 on  10329
Processed  5 on  10329
41911727975.0   dlnvaaz
Processed  6 on  10329
Processed  7 on  10329
13802590014.0   matulik__
Processed  8 on  10329
Processed  9 on  10329
Processed  10 on  10329
Processed  11 on  10329
44194677842.0   every1hatesbenn
Processed  12 on  10329
Processed  13 on  10329
71485223709.0   moha_med.2332009
Processed  14 on  10329
Processed  15 on  10329
3438149810.0   moskri_
Processed  16 on  10329
Processed  17 on  10329
Processed  18 on  10329
14687279054.0   tired_oftwan
Processed  19 on  10329
Processed  20 on  10329
Processed  21 on  10329
Processed  22 on  10329
63393698738.0   aiman.irfaan
Processed  23 on  10329
437981182.0   iss.jas
Processed  24 on  10329
Processed  25 on  10329
Processed  26 on  10329
Processed  27 on  10329
1511055437.0   guntis.u
Processed  28 on  10329
Processed  29 on  10329
72137187185.0   grace.maze12
Processed  30 on  1

KeyboardInterrupt: 

In [ ]:
dfpost

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count,thread_user_pk,caption_text_translated,sentiment_score,sentiment_label
0,3558787428782179764_187264532,3.558787e+18,Behind the scenes,0.0,2025-02-02 01:42:22,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,Behind the scenes,0.997783,POSITIVE
1,3557970465522674878_187264532,3.557970e+18,A little over 24 hours left to vote.\n\n,0.0,2025-01-31 22:39:13,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,A little over 24 hours left to vote.,0.969585,NEGATIVE
2,3557782854629238181_187264532,3.557783e+18,"Galaxy Quest \nLots of cutting holes, playing ...",1.0,2025-01-31 16:26:30,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,"Galaxy Quest \nLots of cutting holes, playing ...",0.868657,NEGATIVE
3,3557776470965660229_187264532,3.557776e+18,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.0,2025-01-31 16:13:49,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.935727,NEGATIVE
4,3556000079513982405_187264532,3.556000e+18,Over due to post videos of my work for @salina...,0.0,2025-01-29 05:24:26,mlssfshn,187264532,0.0,0.0,0.0,6.331078e+10,Over due to post videos of my work for @salina...,0.998091,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135335,3573705187053253481_60152817129,3.573705e+18,Be okay with not telling people EVERYTHING abo...,176.0,1740238878,melanin_moonshine,60152817129,2.0,22.0,NaN,7.082310e+10,NaN,NaN,NaN
135336,3573705005406314979_60152817129,3.573705e+18,29 & I’m really just locking in with this! I d...,1.0,1740238856,melanin_moonshine,60152817129,0.0,0.0,NaN,7.082310e+10,NaN,NaN,NaN
135337,3573703724398712174_60152817129,3.573704e+18,I am going on a nice date 🥰,1.0,1740238703,melanin_moonshine,60152817129,0.0,0.0,NaN,7.082310e+10,NaN,NaN,NaN
135338,3573279294194295350_60152817129,3.573279e+18,Let’s make the matrix crash 😵‍💫😈,2.0,1740188107,melanin_moonshine,60152817129,0.0,0.0,NaN,7.082310e+10,NaN,NaN,NaN
